In [2]:
import os
os.chdir("../")
os.chdir("FinalDataset")

In [3]:
os.getcwd()

'C:\\Users\\SRIKANTH\\Documents\\GitHub\\url_classification_dl\\FinalDataset'

In [4]:
import pandas as pd
import tensorflow as ts
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical

In [23]:
from sklearn.metrics import accuracy_score

In [59]:
from sklearn.preprocessing import MinMaxScaler

In [24]:
m = ts.keras.metrics.CategoricalAccuracy()

In [25]:
from keras.wrappers.scikit_learn import KerasClassifier

from keras.utils import np_utils

from sklearn.model_selection import cross_val_score

from sklearn.model_selection import KFold

In [26]:
data = pd.read_csv("URL_features.csv")

In [27]:
data.drop(columns='Unnamed: 0',inplace=True)
'''
data.replace('Benign_list_big_final',0,inplace=True)
data.replace('DefacementSitesURLFiltered',1,inplace=True)
data.replace('Malware_dataset',2,inplace=True)
data.replace('phishing_dataset',3,inplace=True)
data.replace('spam_dataset',4,inplace=True)
'''
data.replace(True,1,inplace = True)
data.replace(False,0,inplace = True)
y = data["File"]
data.drop(columns = "ext",inplace = True)

In [28]:
data = data.drop(columns = "File")

In [29]:
encoder = LabelEncoder()
encoder.fit(y)
Y = encoder.transform(y)

In [60]:
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(data)
X = pd.DataFrame(X)

In [81]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [82]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding

input_dim = len(data.columns)
model = Sequential()
model.add(Dense(input_dim, input_dim = input_dim , activation = 'relu'))
model.add(Dense(30, activation = 'relu'))
model.add(Dense(25, activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(5, activation = 'softmax'))



model.compile(loss = 'categorical_crossentropy' , optimizer = 'adam' , metrics = ['accuracy',m,f1_m,precision_m, recall_m] )
    

In [83]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, Y, test_size=0.25, random_state=42)

In [94]:
model.fit(X_train,np_utils.to_categorical(y_train),epochs = 50,validation_split=0.3, batch_size = 128)

Epoch 1/50
679/679 [==============================] - 2s 3ms/step - loss: 0.5614 - accuracy: 0.8034 - categorical_accuracy: 0.8034 - f1_m: 0.8000 - precision_m: 0.8362 - recall_m: 0.7672 - val_loss: 0.5772 - val_accuracy: 0.8022 - val_categorical_accuracy: 0.8022 - val_f1_m: 0.7956 - val_precision_m: 0.8263 - val_recall_m: 0.7673
Epoch 2/50
679/679 [==============================] - 2s 3ms/step - loss: 0.5590 - accuracy: 0.8033 - categorical_accuracy: 0.8033 - f1_m: 0.8009 - precision_m: 0.8360 - recall_m: 0.7689 - val_loss: 0.5549 - val_accuracy: 0.8088 - val_categorical_accuracy: 0.8088 - val_f1_m: 0.8052 - val_precision_m: 0.8345 - val_recall_m: 0.7780
Epoch 3/50
679/679 [==============================] - 2s 3ms/step - loss: 0.5589 - accuracy: 0.8047 - categorical_accuracy: 0.8047 - f1_m: 0.8012 - precision_m: 0.8360 - recall_m: 0.7694 - val_loss: 0.5557 - val_accuracy: 0.8038 - val_categorical_accuracy: 0.8038 - val_f1_m: 0.8019 - val_precision_m: 0.8329 - val_recall_m: 0.7733
Epoc

Epoch 26/50
679/679 [==============================] - 2s 3ms/step - loss: 0.5450 - accuracy: 0.8103 - categorical_accuracy: 0.8103 - f1_m: 0.8077 - precision_m: 0.8400 - recall_m: 0.7781 - val_loss: 0.5361 - val_accuracy: 0.8120 - val_categorical_accuracy: 0.8120 - val_f1_m: 0.8110 - val_precision_m: 0.8408 - val_recall_m: 0.7834
Epoch 27/50
679/679 [==============================] - 2s 3ms/step - loss: 0.5426 - accuracy: 0.8114 - categorical_accuracy: 0.8114 - f1_m: 0.8085 - precision_m: 0.8409 - recall_m: 0.7788 - val_loss: 0.5446 - val_accuracy: 0.8084 - val_categorical_accuracy: 0.8084 - val_f1_m: 0.8043 - val_precision_m: 0.8378 - val_recall_m: 0.7736
Epoch 28/50
679/679 [==============================] - 2s 3ms/step - loss: 0.5430 - accuracy: 0.8104 - categorical_accuracy: 0.8104 - f1_m: 0.8085 - precision_m: 0.8409 - recall_m: 0.7788 - val_loss: 0.5451 - val_accuracy: 0.8074 - val_categorical_accuracy: 0.8074 - val_f1_m: 0.8027 - val_precision_m: 0.8379 - val_recall_m: 0.7706
E

In [95]:
y_pred = model.predict(X_test)

In [96]:
predicted = np.argmax(y_pred, axis=1)

In [97]:
from sklearn.metrics import classification_report
accuracy_score(y_test,predicted)

0.8135552222921001

In [98]:
target_names = ['Benign_list_big_final','DefacementSitesURLFiltered','Malware_dataset','phishing_dataset','spam_dataset']
print(classification_report(y_test, predicted, target_names=target_names))

                            precision    recall  f1-score   support

     Benign_list_big_final       0.81      0.84      0.82      8903
DefacementSitesURLFiltered       0.84      0.93      0.88     23970
           Malware_dataset       0.80      0.52      0.63      2864
          phishing_dataset       0.46      0.24      0.31      2493
              spam_dataset       0.72      0.61      0.66      3112

                  accuracy                           0.81     41342
                 macro avg       0.73      0.63      0.66     41342
              weighted avg       0.80      0.81      0.80     41342



In [103]:
model.save("nn1")

INFO:tensorflow:Assets written to: nn1\assets
